In [1]:
%%capture
import pandas as pd
import numpy as np
import pyemd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import nltk
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from nltk import tokenize    
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')
    
def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    
    patterns = [word_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token.lower())
        words.extend(tokens)
    return words

### carregando modelo pré-treinado e nomalizando-o
model = KeyedVectors.load_word2vec_format("wiki_vectors_format_without_stopwords.bin", binary=False)
model.init_sims(replace=True) 

def headline_embeddings(headline):
    emb = []
    for word in headline:
        emb.append(model[word])
    return emb
        
headline = "O que Bolsonaro e Moro colocam em jogo"
headline = parse(headline)
hl_e1 = headline_embeddings(headline)

AttributeError: module 'gensim' has no attribute 'downloader'

In [2]:
headline2 = "Os prejuizos da Operacao Lava Jato para o Brasil"
headline2 = parse(headline2)
hl_e2 = headline_embeddings(headline2)

from math import inf
from math import sqrt

def calc_wmd(vector1, vector2):
    wmd = 0     
    
    assert len(vector1) == len(vector2)
    
    for w in vector1:
        min_dist = inf
        for w2 in vector2:
            min_dist = min(min_dist, sqrt(sum((w - w2) ** 2)))
        
        wmd += min_dist * 1.0/len(vector1) ### normalize score
        
    return wmd

calc_wmd(hl_e1, hl_e2)
model.wmdistance(headline, headline2)

NameError: name 'headline_embeddings' is not defined

In [ ]:
docs = ["Selecao brasileira joga proxima terca-feira",
        "Neymar é acusado de estupro",
        "Mitos e verdades sobre o consumo de pimenta para a saude",
        "Saiba qual é o tecido que os gatos mais gostam de arranhar",
        "Lava Jato desconfiava de Leo Pinheiro ate versao do triplex do Guaruja",
        "China e EUA concordam em retomar negociacoes comerciais",
        "As gambiarras que todo jogador ja fez na vida",
        "O seda em fim de ciclo com vendas absurdas",
        "Os prejuizos da Operacao Lava Jato para o Brasil",
        "O que Bolsonaro e Moro colocam em jogo"]

processed_docs = [parse(w) for w in docs]

from gensim.similarities import WmdSimilarity
num_best = 3
instance = WmdSimilarity(processed_docs, model, num_best)

query =  parse("Bolsonaro e Trump reforcam lacos diplomaticos")
sim = instance[query]

for i in range(num_best):
    print ('Similaridade = %.4f' % sim[i][1])
    print (docs[sim[i][0]])

#### Referencias 
[1]https://docs.google.com/presentation/d/1CTmpRHA2gurNM6APTpXzmpfVSZLgRoOszA3XQ-PyCto/edit#slide=id.gc6f73a04f_0_0
[2]https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html